In [1]:
import pandas as pd
import numpy as np
import shapely
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout, UpSampling2D, Reshape
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
df = pd.read_pickle('./final.pkl')
mb_am_temp = 56
mb_af_temp = 63
mb_pm_temp = 60

In [3]:
# Function to map AMafPM to ref_temp
def map_to_ref_temp(row):
    if row['AMafPM'] == [1,0,0]:
        return 56
    elif row['AMafPM'] == [0,1,0]:
        return 63
    elif row['AMafPM'] == [0,0,1]:
        return 60
    else:
        return None  # or a default value

# Apply the function to create the new column
df['ref_temp'] = df.apply(map_to_ref_temp, axis=1)

In [4]:
landuse_dim, water_dim, natural_dim, highway_dim  = [len(df.iloc[0,:]['feature_grids'][x][0][0]) for x in df.iloc[0,:]['feature_grids'].keys()]

In [5]:
df['landuse']=df['feature_grids'].apply(lambda x: x['landuse'])
df['water']=df['feature_grids'].apply(lambda x: x['water'])
df['natural']=df['feature_grids'].apply(lambda x: x['natural'])
df['highway']=df['feature_grids'].apply(lambda x: x['highway'])


In [6]:
scaler = MinMaxScaler()

# Find global min and max
global_min = np.min([np.min(subgrid) for subgrid in df["Subgrid"]])
global_max = np.max([np.max(subgrid) for subgrid in df["Subgrid"]])

# Fit the scaler using global min and max
scaler.fit(np.array([[global_min], [global_max]]))

# Apply the same scaling to each subgrid
scaled_subgrids = []

for subgrid in df["Subgrid"]:
    scaled_subgrid = scaler.transform(subgrid.flatten().reshape(-1, 1))
    scaled_subgrids.append(scaled_subgrid.reshape(subgrid.shape))

# Assign scaled subgrids back to DataFrame
df["Subgrid"] = pd.Series(scaled_subgrids, index=df.index)

In [7]:
def rotate_and_augment(df):
    # Creating an empty DataFrame to store augmented data
    augmented_data = pd.DataFrame()
    
    # Iterating through each row in the original DataFrame
    for _, row in df.iterrows():
        # Storing original and rotated versions of each grid in a list
        all_versions = []
        
        # Original data
        original_data = {
            "Subgrid": row["Subgrid"],
            "height_grid": row["height_grid"],
            "landuse": row["landuse"],
            "water": row["water"],
            "natural": row["natural"],
            "highway": row["highway"],
            # ... include other non-rotated columns as needed
            "AMafPM": row["AMafPM"],
            "ref_temp": row["ref_temp"]
        }
        all_versions.append(original_data)
        
        # Creating rotated versions
        for angle in [90, 180, 270]:
            rotated_data = {
                "Subgrid": np.rot90(row["Subgrid"], k=angle//90),
                "height_grid": np.rot90(row["height_grid"], k=angle//90),
                "landuse": np.rot90(row["landuse"], k=angle//90, axes=(0, 1)),
                "water": np.rot90(row["water"], k=angle//90, axes=(0, 1)),
                "natural": np.rot90(row["natural"], k=angle//90, axes=(0, 1)),
                "highway": np.rot90(row["highway"], k=angle//90, axes=(0, 1)),
                # ... include other non-rotated columns as needed
                "AMafPM": row["AMafPM"],
                "ref_temp": row["ref_temp"]
            }
            all_versions.append(rotated_data)
        
        # Appending all versions to the augmented_data DataFrame
        augmented_data = augmented_data.append(all_versions, ignore_index=True)
    
    return augmented_data
df = rotate_and_augment(df)

/tmp/ipykernel_12394/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_12394/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_12394/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_12394/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_1

In [8]:
from sklearn.model_selection import train_test_split
test_size_ratio = 0.2  # Here, 20% of the data goes to the test set.

# Split the data
train_df, test_df = train_test_split(df, test_size=test_size_ratio, random_state=42)

In [9]:
def into_mlp (key,mode="train"):
    if mode == "train":
        feature = train_df[key]
    elif mode == "test":
        feature = test_df[key]
    shp = feature.iloc[0].shape
    rows = feature.shape[0]
    vals = int(np.prod([shp[i] for i in range(len(shp))]))
    return np.array(feature.tolist()).reshape(rows,vals)

In [10]:
height_grid = into_mlp('height_grid')
landuse = into_mlp('landuse')
water = into_mlp('water')
natural = into_mlp('natural')
highway = into_mlp('highway')
AMafPM = np.array(train_df['AMafPM'].tolist())
ref_temp = np.array(train_df['ref_temp'].tolist())
target = into_mlp('Subgrid')

In [11]:
height_grid_test = into_mlp("height_grid",mode="test")
landuse_test = into_mlp("landuse",mode="test")
water_test = into_mlp('water',mode="test")
natural_test = into_mlp('natural',mode="test")
highway_test = into_mlp('highway',mode="test")
AMafPM_test = np.array(test_df['AMafPM'].tolist())
ref_temp_test = np.array(test_df['ref_temp'].tolist())
target_test = into_mlp('Subgrid',mode="test")

In [12]:
target.shape

(2035, 4096)

In [13]:
p_ftrs = []
for ftr in [height_grid, landuse, water, natural, highway, AMafPM, ref_temp]:
    if len(ftr.shape) == 1:
        p_ftrs.append(np.expand_dims(ftr,axis=1))
    else:
        p_ftrs.append(ftr)
concatenated_features = np.concatenate(p_ftrs,axis=1)

In [14]:
concatenated_features.shape

(2035, 94212)

In [15]:
from keras.models import Sequential
from keras.layers import Dense

# Define MLP architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(concatenated_features.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(4096, activation='relu'))  # Adjust output layer according to your task

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(concatenated_features, target, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
51/51 [==============================] - 2s 15ms/step - loss: 2.8860 - accuracy: 6.1425e-04 - val_loss: 1.1098 - val_accuracy: 0.0000e+00
Epoch 2/100
51/51 [==============================] - 0s 6ms/step - loss: 0.9452 - accuracy: 0.0000e+00 - val_loss: 0.7914 - val_accuracy: 0.0000e+00
Epoch 3/100
51/51 [==============================] - 0s 6ms/step - loss: 0.7577 - accuracy: 0.0000e+00 - val_loss: 0.7224 - val_accuracy: 0.0000e+00
Epoch 4/100
51/51 [==============================] - 0s 6ms/step - loss: 0.7236 - accuracy: 0.0000e+00 - val_loss: 0.7127 - val_accuracy: 0.0000e+00
Epoch 5/100
51/51 [==============================] - 0s 6ms/step - loss: 0.7136 - accuracy: 0.0000e+00 - val_loss: 0.7128 - val_accuracy: 0.0000e+00
Epoch 6/100
51/51 [==============================] - 0s 6ms/step - loss: 0.7070 - accuracy: 0.0012 - val_loss: 0.7127 - val_accuracy: 0.0000e+00
Epoch 7/100
51/51 [==============================] - 0s 6ms/step - loss: 0.7014 - accuracy: 0.0000e+00 - val_

Epoch 56/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6788 - accuracy: 0.0000e+00 - val_loss: 0.6778 - val_accuracy: 0.0000e+00
Epoch 57/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6786 - accuracy: 6.1425e-04 - val_loss: 0.6779 - val_accuracy: 0.0000e+00
Epoch 58/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6785 - accuracy: 0.0000e+00 - val_loss: 0.6781 - val_accuracy: 0.0000e+00
Epoch 59/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6786 - accuracy: 0.0000e+00 - val_loss: 0.6777 - val_accuracy: 0.0000e+00
Epoch 60/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6786 - accuracy: 0.0000e+00 - val_loss: 0.6784 - val_accuracy: 0.0000e+00
Epoch 61/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6786 - accuracy: 6.1425e-04 - val_loss: 0.6778 - val_accuracy: 0.0000e+00
Epoch 62/100
51/51 [==============================] - 0s 6ms/step - loss: 0.6785 - accuracy: 6.1425e

In [16]:
p_ftrs = []
for ftr in [height_grid_test, landuse_test, water_test, natural_test, highway_test, AMafPM_test, ref_temp_test]:
    if len(ftr.shape) == 1:
        p_ftrs.append(np.expand_dims(ftr,axis=1))
    else:
        p_ftrs.append(ftr)
concatenated_features_test = np.concatenate(p_ftrs,axis=1)

In [17]:
predictions = model.predict(concatenated_features_test)

16/16 [==============================] - 0s 3ms/step


In [18]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(target_test, predictions)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

In [19]:
mse = mse * (global_max - global_min)

In [21]:
print("MLP",mse,rmse)

MLP 1.2412429 0.24120286
